# Reading data from adc_benchmark

In [1]:
# Import some useful libraries
import serial, time
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [18, 8]

## First, open the serial connection

In [5]:
ser = serial.Serial('/dev/ttyACM1', 115200) # Usually ACM0, changes with restarts

## Now we send commands and read the data.

To send a command
    - clear the buffer
    - write command with ser.write('c')
    - wait for a response.

To read back the response for a short response, like the time taken message returned by 't'
    - Use ser.read_all()
    
To read back the response when the micro needs to send lots of data
    - Use ser.readline() or ser.read() in a loop. If you read fast enough, you can empty the buffer prematurely so
    - check if the bufer is empty, and if it is, wait and check again before exiting the loop. If not
    - Keep reading until the buffer stays empty


# Reading one value:

In [13]:
r = ser.read_all() # clear serial buffer
ser.write(b'a')
while ser.in_waiting < 1:
    pass # wait for a response
time.sleep(0.05) # This extra delay helps with reliability - it gives the micro time to send all it needs to
r = ser.read_all()
print(r) # Prints the value

b'26857\r\n'


# Reading and sending 20k samples

In [35]:

# Get the values
ser.write(b'd') # See code for command meanings
while ser.in_waiting < 1:
    pass # wait for a response

a = b'' # Keep bytes as a string
i = 0
while True:
    if ser.in_waiting < 1:
        time.sleep(0.005) # Wait and check again
        if ser.in_waiting < 1:
            break
    b = ser.read_all() # Using read() is much slower due to extra delays
    a += b
    i += 1
    
print(len(a)) # Should be 20000 readings, 2 bytes per reading (so 40k bytes)

40000


In [36]:
# How long did it take?
r = ser.read_all() # clear serial buffer
ser.write(b't') # <<<< gets time
while ser.in_waiting < 1:
    pass # wait for a response
time.sleep(0.05) # This extra delay helps with reliability - it gives the micro time to send all it needs to
r = ser.read_all()
print(r) # Prints the time

b'123471\r\n'


In [38]:
(123471/20000) # 6.17 us / sample

6.17355

In [39]:
1000000/(123471/20000) # ~162ksps

161981.35594593064

converting to numbers:

In [40]:
a[:50] # This is what the result string looks like

b'`\x89R\xd5g6ikb\xa4o\xbcd\xf0j\xfds\xe6}\xbbw\xbbv\xd2sP}.\x84\xb6\x84\xf0\x85@}\xb0w\x19x\x9br3t\x17w\xdbn\xbfm\x7f'

In [48]:
vals = []
for i in range(int(len(a)/2)):
    val = int.from_bytes(a[i*2:i*2+1], byteorder='big') # Check byteorder (test with known signal)
    vals.append(val)
vals[:5] # look at just the first 5

[96, 82, 103, 105, 98]

In [49]:
len(vals)

20000

# Reading 20k samples, then sending

In [52]:
# Get the values
ser.write(b'f') # See code for command meanings
while ser.in_waiting < 1:
    pass # wait for a response

a = b'' # Keep bytes as a string
i = 0
while True:
    if ser.in_waiting < 1:
        time.sleep(0.005) # Wait and check again
        if ser.in_waiting < 1:
            break
    b = ser.read_all() # Using read() is much slower due to extra delays
    a += b
    i += 1
    
print(len(a)) # Should be 20000 readings, 2 bytes per reading (so 40k bytes)

40000


In [53]:
# How long did it take?
r = ser.read_all() # clear serial buffer
ser.write(b't') # <<<< gets time
while ser.in_waiting < 1:
    pass # wait for a response
time.sleep(0.05) # This extra delay helps with reliability - it gives the micro time to send all it needs to
r = ser.read_all()
print(r) # Prints the time

b'101434\r\n'


In [55]:
1000000/(101434/20000) # ~200ksps

197172.54569473746

### Speed has gone from ~163ksps to ~197 ksps by reading first, then sending

# Reading from different analog pins

In [56]:
# Get the values
ser.write(b'g') # See code for command meanings
while ser.in_waiting < 1:
    pass # wait for a response

a = b'' # Keep bytes as a string
i = 0
while True:
    if ser.in_waiting < 1:
        time.sleep(0.005) # Wait and check again
        if ser.in_waiting < 1:
            break
    b = ser.read_all() # Using read() is much slower due to extra delays
    a += b
    i += 1
    
print(len(a)) # Should be 20000 readings, 2 bytes per reading (so 40k bytes)

40000


In [57]:
# How long did it take?
r = ser.read_all() # clear serial buffer
ser.write(b't') # <<<< gets time
while ser.in_waiting < 1:
    pass # wait for a response
time.sleep(0.05) # This extra delay helps with reliability - it gives the micro time to send all it needs to
r = ser.read_all()
print(r) # Prints the time

b'91624\r\n'


In [58]:
1000000/(91624/20000) # ~200ksps

218283.41919147823

In [59]:
# Reading from different pins is actually a little faster, since some use ADC1 instead of ADC0 internally.

In [6]:
ser.close() # Close the connection when you're done.